## Function ##
We often want to take a piece of code and wrap it into a function. That way we can use the code in several places without 
repeating it.

### Function Parameters ###

In [1]:
def add(a,b):
    return a+b
print add(32,5)
print add('in','out')

37
inout


Functions can have parameters with default value. This helps keep the interface the the function simple while at the same time allowing fine control when needed. It also help keep functions backward compatible with code that was  written before the new parameters were added.

In [2]:
def paren(String,left='(',right=')'):
    print left+String+right
paren('cat')
paren('dog','[',']')
paren('this',right=' and that)')


(cat)
[dog]
(this and that)


Function can even have generic parameters, which allow functions to have an arbitrary number of parameters and an arbitrary set of keywards. 

(*Advanced topic*: Another cool thing that this allows is to define a generic "wrapper" function which can be wrapped around any function and change it's behaviour. For more on that pattern google `"python decorators"`)

In [3]:
def generic(*args,**kwargs):
    print 'args=[',','.join([str(arg) for arg in args]),']'
    print 'kwargs={',','.join(['(%s=%s)'%(str(k),str(kwargs[k])) for k in kwargs.keys()]),'}'
generic(1,2,3,4)
generic('this','that',butnot='the other')

args=[ 1,2,3,4 ]
kwargs={  }
args=[ this,that ]
kwargs={ (butnot=the other) }


### Namespaces and scopes ###

A namespace is a collection of names that are accessible to a program. As we saw when we discussed the import command,
it is often better to import functions into a separate namespace as when we use 

`import pandas as pd`

which creates names of the form `pd.DataFrame` rather than

`from pandas import *`

which creates names of the form `DataFrame`.

An important property of good software design is that function not have **side effects**. In other words, the only effect that calling a function would have on the state of the variables in the caller is done via assignment of the returned value to
a variable in the calling function. Any other kind of impact is called a "side-effect" and is undesirable because it breaks the logical modularization of the code.

On common side-effect can occur when a function assigns a value to a variable that is defined in the calling program. To protect against this side effect the variables that are defined inside a function occupy a separate name space, called the **local** name space. As in python variables come into being when they are assigned to, a variable is declared local if it is assigned a value inside the program. In addition, the parameters to the function are part of the local namespace.

The local namespace is deleted when the function returns.

Note however that the function **can read** variables that are defined in the calling program. It just can't **write** to those variables. Reading external variables gives the function access to global variables without requiring that they are passed as parameters.

In [4]:
# an example of local and global vaiables.
def f():
    print 'initially b=',b
    a=2   # create a local variable 'a' that masks the global 'a'
    print 'finally  a=',a,' b=',b
a=5; b=10
f()
print "from caller a=",a,' b=',b

initially b= 10
finally  a= 2  b= 10
from caller a= 5  b= 10


#### Guess and check what happens if you add the variable 'a' to the command "print 'initially b=',b"

#### locals and globals are dictionaries
All variables that are defined in the current context, either as global or as local variables, can be accessed through the dictionaries named `local` and `global`

In [5]:
def f(x,y=1):
    print 'locals:'
    print '\n'.join([str(item) for item in locals().items()]);
    print 'globals:'
    print '\n'.join(['\t'+str(key)+':'+str(value) for key,value in globals().items()]);
f(5);

locals:
('y', 1)
('x', 5)
globals:
	_dh:[u'/Users/yoavfreund/academic.papers/Courses/DSE200/DSE200-notebooks-2015/day_4_OOP_and_matplotlib/OOP']
	__:
	_i:# an example of local and global vaiables.
def f():
    print 'initially b=',b
    a=2   # create a local variable 'a' that masks the global 'a'
    print 'finally  a=',a,' b=',b
a=5; b=10
f()
print "from caller a=",a,' b=',b
	quit:<IPython.core.autocall.ZMQExitAutocall object at 0x1056de250>
	__builtins__:<module '__builtin__' (built-in)>
	_ih:['', u"def add(a,b):\n    return a+b\nprint add(32,5)\nprint add('in','out')", u"def paren(String,left='(',right=')'):\n    print left+String+right\nparen('cat')\nparen('dog','[',']')\nparen('this',right=' and that)')", u"def generic(*args,**kwargs):\n    print 'args=[',','.join([str(arg) for arg in args]),']'\n    print 'kwargs={',','.join(['(%s=%s)'%(str(k),str(kwargs[k])) for k in kwargs.keys()]),'}'\ngeneric(1,2,3,4)\ngeneric('this','that',butnot='the other')", u'# an example of local and g

### Lambda expressions ###

Functions can be parameters to other functions. For examples the `sort` command has an optional parameter `key`
which defines a function whose input is an item in the list to be sorted and whose output sortable element such as a number or a string.

Such functions are often very simple. Writing these function explicitly can lead to bluky and confusing code.

A better solution is to use what are called "anonymous functions" or "lambda functions". These are functions which do not have a name and whose definition is very short and can be included inside the call to the function. See the example below.



In [6]:
student_tuples = [
        ('john', 'A', 15),
        ('jane', 'B', 12),
        ('dave', 'B', 10),
]

def keyfunction(student):
    return student[2]
print sorted(student_tuples, key=keyfunction)   # sort by age

# is equivalent to 
print sorted(student_tuples, key=lambda student: student[2])   # sort by age



[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]
[('dave', 'B', 10), ('jane', 'B', 12), ('john', 'A', 15)]


## Object oriented programming ##

The code in this notebook is associated with the book ["Introduction to computation
and programming using Python" By John V. Guttag](http://mitpress.mit.edu/books/introduction-computation-and-programming-using-python-0) The code here is taken from Chapter 8

While functions help modularize computer code they leave much to be desired. It is often desirable to create modules that combine several functions together with the data on which these functions operate. In python these modules are called **classes**. A class is a definition of a type of **object** which can be manipulated in a predefined set of ways.

### A new class: IntSet
Below we define a class called `IntSet` which represents a set of integers. The commands to create two new empty sets 
of type `intSet` called `X` and `Y` are

> X=IntSet(); Y=IntSet()

An object is a collection of **attributes** and **methods**. The attributes store the data which defines the object while the methods are the functions that can operate on this data.

The syntax for accessing the attribute `vals` in the variable `X` is `X.vals`. The syntax for calling the method `insert` to insert the number `7` into the intSet Y is `Y.insert(7)`

By Convention, if the name of an attribute or a method start with an underscore `_` the attribute or method is considered **private**. This means that the methods and attributes should not be accessed directly by outside code.

Two standard private methods are:

* `__init__`  a method for initializing the object. This method is called automatically when the class name is called.
* `__str__` a method that returns a representation of the object as a string. This method is called by the print command.

In [ ]:
#Page 93, Figure 8.1
class IntSet(object):
    """An intSet is a set of integers"""
    #Information about the implementation (not the abstraction)
    #The value of the set is represented by a list of ints, self.vals.
    #Each int in the set occurs in self.vals exactly once.
    
    def __init__(self):
        """Create an empty set of integers"""
        self.vals = []

    def insert(self, e):
        """Assumes e is an integer and inserts e into self"""
        if not e in self.vals:
            self.vals.append(e)

    def member(self, e):
        """Assumes e is an integer
           Returns True if e is in self, and False otherwise"""
        return e in self.vals

    def remove(self, e):
        """Assumes e is an integer and removes e from self
           Raises ValueError if e is not in self"""
        try:
            self.vals.remove(e)
        except:
            raise ValueError(str(e) + ' not found')

    def getMembers(self):
        """Returns a list containing the elements of self.
           Nothing can be assumed about the order of the elements"""
        return self.vals[:]

    def __str__(self):
        """Returns a string representation of self"""
        self.vals.sort()
        result = ''
        for e in self.vals:
            result = result + str(e) + ','
        return '{' + result[:-1] + '}' #-1 omits trailing comma

In [ ]:
#Page 94
s = IntSet()
print 's=',s
s.insert(3)
print s.member(3)

### An exisiting class: arrays
The package **numpy** defines the type "array" which defines k-dimensional arrays and over-rides many of the operations for them. A special method name is associated with each operand, some of those are:

|operator| method name |
|--------|-------------|
| `+`    |   `__add__` |  

In [7]:
import numpy
#Page 148
a1 = numpy.array([1, 2, 4])
print 'a1 =', a1
a2 = a1*2
print 'a2 =', a2
print 'a1 + 3 =', a1 + 3
print '3 - a1 =', 3 - a1
print 'a1 - a2 =', a1 - a2
print 'a1*a2 =', a1*a2

a1 = [1 2 4]
a2 = [2 4 8]
a1 + 3 = [4 5 7]
3 - a1 = [ 2  1 -1]
a1 - a2 = [-1 -2 -4]
a1*a2 = [ 2  8 32]


### Defining the "Person" class ###

We define a base class which holds the methods (functions) and properties (data) commong to all persons.

In [ ]:
#Page 97, Figure 8.2
import datetime

class Person(object):  # (object) is redundant, all classes inherit from "object".

    def __init__(self, name):
        """Create a person"""
        self.name = name
        try:
            lastBlank = name.rindex(' ')
            self.lastName = name[lastBlank+1:]
        except:  # exception occurs if cannot find a space and something following it.
            self.lastName = name
        self.birthday = None
 
    def getName(self):
        """Returns self's full name"""
        return self.name

    def getLastName(self):
        """Returns self's last name"""
        return self.lastName

    def setBirthday(self, birthdate):
        """Assumes birthdate is of type datetime.date
           Sets self's birthday to birthdate"""
        self.birthday = birthdate

    def getAge(self):
        """Returns self's current age in days"""
        if self.birthday == None:
            raise ValueError
        return (datetime.date.today() - self.birthday).days

    def __lt__(self, other):
        """Returns True if self's name is lexicographically
           less than other's name, and False otherwise"""
        if self.lastName == other.lastName:
            return self.name < other.name
        return self.lastName < other.lastName

    def __str__(self):
        """Returns self's name"""
        return self.name

In [ ]:
#Page 97
me = Person('Michael Guttag')
him = Person('Barack Hussein Obama')
her = Person('Madonna')
print him.getLastName()
him.setBirthday(datetime.date(1961, 8, 4))
her.setBirthday(datetime.date(1958, 8, 16))
print him.getName(), 'is', him.getAge(), 'days old'

### Objects of type Person can be used anywhere ###
We can create a list of people. As we defined the order relationship between persons `__lt__`, we can sort the list.


In [ ]:
#Page 98
print type(me)
pList = [me, him, her]
for p in pList:
    print p
pList.sort()
for p in pList:
    print p

### Defining MITPerson as a sub-class of Person ###

In [ ]:
#Page 99, Figure 8.3
class MITPerson(Person):
    
    nextIdNum = 0   #identification number - this is a class variable, 
                    # is attached to the whole class, not to each object.
    
    def __init__(self, name):
        Person.__init__(self, name)  # call the parent class initializer.
        self.idNum = MITPerson.nextIdNum
        MITPerson.nextIdNum += 1
        
    def getIdNum(self):
        return self.idNum
    
    def __lt__(self, other):
        return self.idNum < other.idNum

    def isStudent(self):
        return isinstance(self, Student)

In [ ]:
#Page 100
p1 = MITPerson('Barbara Beaver')
print str(p1) + '\'s id number is ' + str(p1.getIdNum())

### Excercise:
Over-ride the method `__str__` for `MITPerson` so that the default printout of an `MITPerson` includes the fact that they are from MIT and their ID number.

In [ ]:
p1 = MITPerson('Mark Guttag')
p2 = MITPerson('Billy Bob Beaver1')   # three different persons (person objects) called Billy Bob Beaver.
p3 = MITPerson('Billy Bob Beaver2')
p4 = Person('Billy Bob Beaver3')
print 'variable type is MITPerson',[type(p) is MITPerson for p in [p1,p2,p3,p4]]

print 'p1 < p2 =', p1 < p2
print 'p3 < p2 =', p3 < p2
print 'p4 < p1 =', p4 < p1

print 'p1 < p4 =', p1 < p4

### Fix this error ###
1. What cased the error? Why is `p4<p1` ok, but `p1<p4` not ok?
2. How would you fix the problem?

### Next, we create Classes Student, UG and Grad ###

In [ ]:
#Page 101, Figure 8.4
class Student(MITPerson):
    pass

class UG(Student):
    def __init__(self, name, classYear):
        MITPerson.__init__(self, name)
        self.year = classYear
    def getClass(self):
        return self.year
    
class Grad(Student):
    pass

### Type vs. isinstance
`Grad` and `UG` are subclasses of `Student`.
If a an object is initialized as a `Grad` it's type is `Grad`. To check the ancestors of the type
we use the command `issinstance`. Thus calling `isinstance` on an object of type `UG` will return **True** on the classes `UG,Student,MITPerson,Person` but will return false on the class `Grad`

In [ ]:
#Page 101
p5 = Grad('Buzz Aldrin')
print p5, 'is a graduate student is', type(p5) == Grad
print p5, 'is an undergraduate student is', type(p5) == UG
print p5, 'is a student is',type(p5)==Student

In [ ]:
#Page 102
p6 = UG('Billy Beaver', 1984)
p7 = Student('Eternal Student')
def isStudent(self):
    return isinstance(self, Student)
print p5, 'is a type of student is', p5.isStudent()
print p6, 'is a type of student is', p6.isStudent()
print p3, 'is a type of student is', p3.isStudent()
print p7, 'is a type of student is', p7.isStudent()

In [ ]:
# one more type of student.

class TransferStudent(Student):

    def __init__(self, name, fromSchool):
        MITPerson.__init__(self, name)
        self.fromSchool = fromSchool

    def getOldSchool(self):
        return self.fromSchool

### Define the class `Grades` which defines a grade list

In [ ]:
#Page 103, Figure 8.5

class Grades(object):
    """A mapping from students to a list of grades"""
    def __init__(self):
        """Create empty grade book"""
        self.students = []
        self.grades = {}
        self.isSorted = True

    def addStudent(self, student):
        """Assumes: student is of type Student
           Add student to the grade book"""
        if student in self.students:
            raise ValueError('Duplicate student')
        self.students.append(student)
        self.grades[student.getIdNum()] = []
        self.isSorted = False

    def addGrade(self, student, grade):
        """Assumes: grade is a float
           Add grade to the list of grades for student"""
        try:
            self.grades[student.getIdNum()].append(grade)
        except:
            raise ValueError('Student not in mapping')

    def getGrades(self, student):
        """Return a list of grades for student"""
        try: #return copy of student's grades
            return self.grades[student.getIdNum()][:]
        except:
            raise ValueError('Student not in mapping')

    def getStudents(self):
        """Return a list of the students in the grade book"""
        if not self.isSorted:
            self.students.sort()
            self.isSorted = True
        return self.students[:] #return copy of list of students

In [ ]:
#Page 105, Figure 8.6

# A function (not a method) that takes as input a "Grades" object and returns 
# the average grade of each student.
def gradeReport(course):
    """Assumes course is of type Grades"""
    report = ''
    for s in course.getStudents():
        tot = 0.0
        numGrades = 0
        for g in course.getGrades(s):
            tot += g
            numGrades += 1
        try:
            average = tot/numGrades
            report = report + '\n'\
                     + str(s) + '\'s mean grade is ' + str(average)
        except ZeroDivisionError:
            report = report + '\n'\
                     + str(s) + ' has no grades'
    return report

ug1 = UG('Jane Doe', 2014)
ug2 = UG('John Doe', 2015)
ug3 = UG('David Henry', 2003)
g1 = Grad('Billy Buckner')
g2 = Grad('Bucky F. Dent')
sixHundred = Grades()
sixHundred.addStudent(ug1)
sixHundred.addStudent(ug2)
sixHundred.addStudent(g1)
sixHundred.addStudent(g2)
for s in sixHundred.getStudents():
    sixHundred.addGrade(s, 75)
sixHundred.addGrade(g1, 25)
sixHundred.addGrade(g2, 100)
sixHundred.addStudent(ug3)
print gradeReport(sixHundred)

In [ ]:
#Page 107, Figure 8,7
# A generator function
def getStudents(self):
    """Return the students in the grade book one at a time"""
    if not self.isSorted:
        self.students.sort()
        self.isSorted = True
    for s in self.students:
        yield s

In [ ]:
#Page 107
book = Grades()
book.addStudent(Grad('Julie'))
book.addStudent(Grad('Charlie'))
for s in book.getStudents():
    print s